In [0]:
%sql
create or replace temp view prev as(
select 
date_format(f.DATA_COMPRA,'yyyy-MM') as ANOMES
,sum(f.QUANTIDADE) as UND
from dmn_transformacao_digital_dev.db_analytics.cpv_fvendas as f
left join dmn_transformacao_digital_dev.db_analytics.cpv_dprodutos as dp on dp.ID_PRODUTO = f.ID_PRODUTO
left join dmn_transformacao_digital_dev.db_analytics.cpv_dpdv as dl on dl.ID_LOJA = f.ID_LOJA
Where dl.GRUPO = "DPSP" and f.DATA_COMPRA >= "2021-01-01" and f.DATA_COMPRA <= "2025-02-28"
group by all

)

In [0]:
!pip install --upgrade pip
!pip install python-slugify loguru openpyxl scikit-learn joblib shap imbalanced-learn

In [0]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from loguru import logger

In [0]:
# Função para pré-processar os dados
def preprocess_data(df):
    logger.info("Iniciando pré-processamento...")
    df = df.sort_values('ANOMES').reset_index(drop=True)
    df = df.dropna().copy()  # Garante que não altera a versão original por acidente
    if 'MES_SEQ' not in df.columns:
        df['MES_SEQ'] = np.arange(len(df))
    else:
        logger.warning("'MES_SEQ' já existia! Substituindo valores.")
        df['MES_SEQ'] = np.arange(len(df))
    return df
  

# Função para treinar o modelo reg linear
def train_model(X, Y):
    logger.info("Treinando modelo...")
    rl = LinearRegression()
    rl.fit(X,Y)
    logger.success("Modelo treinado.")
    return rl

# Função para treinar o modelo Random Forest
def train_random_forest_model(X, Y):
    logger.info("Treinando modelo Random Forest...")
    rf = RandomForestRegressor(
        n_estimators=300,  # Número de árvores
        max_depth=10,    # Deixa as árvores crescerem até necessário
        random_state=42,    # Para resultados reproduzíveis
        max_features= 'sqrt',
        min_samples_leaf =1,
        min_samples_split=2
        
    )
    rf.fit(X, Y)
    logger.success("Modelo Random Forest treinado.")
    return rf

# Função do previsor
def previsor(modelo, df):
    logger.info("calculando a previsão...")
    p = df['MES_SEQ'].max() + 1
    X_NOVO = np.array([[p]])
    Y_NOVO = modelo.predict(X_NOVO)


    logger.success(f"Previsao: {Y_NOVO}")
    
    return Y_NOVO

# Função para avaliar o modelo
def avaliar_modelo(modelo, df):
    logger.info("Avaliando modelo...")
    
    
    X = df['MES_SEQ'].values.reshape(-1, 1)
    y_true = df['UND']
    y_pred = modelo.predict(X)

    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)

    logger.success(f"Avaliação concluída. R²: {r2:.4f} | MAE: {mae:.2f}")
  
    return r2, mae



In [0]:
# Pipeline de execução: Leitura, treino, validação, avaliação e salvamento

# 1. Ler os dados
aprendizagem = spark.table("prev").toPandas()

# 2. Pre-tratamento
aprendizagem = preprocess_data(aprendizagem)

# 3. Treinar o modelo
modelo_rf = train_random_forest_model(
    aprendizagem['MES_SEQ'].values.reshape(-1, 1), 
    aprendizagem['UND']
)

# 4. criar a previsão
previsao = previsor(modelo_rf, aprendizagem)

#5. Avalaiação do modelo
nota = avaliar_modelo(modelo_rf,aprendizagem)



2025-04-29 14:41:18.964 | INFO     | __main__:preprocess_data:3 - Iniciando pré-processamento...
2025-04-29 14:41:18.968 | INFO     | __main__:train_random_forest_model:24 - Treinando modelo Random Forest...
2025-04-29 14:41:19.353 | SUCCESS  | __main__:train_random_forest_model:35 - Modelo Random Forest treinado.
2025-04-29 14:41:19.355 | INFO     | __main__:previsor:40 - calculando a previsão...
2025-04-29 14:41:19.382 | SUCCESS  | __main__:previsor:46 - Previsao: [87752.97]
2025-04-29 14:41:19.383 | INFO     | __main__:avaliar_modelo:52 - Avaliando modelo...
2025-04-29 14:41:19.412 | SUCCESS  | __main__:avaliar_modelo:62 - Avaliação concluída. R²: 0.9389 | MAE: 3934.31


In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Define o modelo base
rf = RandomForestRegressor(random_state=42)

# Define o grid de hiperparâmetros para testar
param_grid = {
    'n_estimators': [100, 200, 300],  # número de árvores
    'max_depth': [5, 10, 15, None],   # profundidade máxima
    'min_samples_split': [2, 5, 10],  # mínimo para dividir
    'min_samples_leaf': [1, 2, 4],    # mínimo em folha
    'max_features': ['auto', 'sqrt']  # número de variáveis
}

# Configura o GridSearch
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,   # 5-fold cross validation
    scoring='neg_mean_absolute_error',  # queremos minimizar o erro absoluto
    n_jobs=-1,  # usa todos os núcleos da CPU
    verbose=2   # mostra o progresso
)

# Prepara os dados
X = aprendizagem['MES_SEQ'].values.reshape(-1, 1)
y = aprendizagem['UND']

# Roda a busca
grid_search.fit(X, y)

# Melhor modelo encontrado
melhor_modelo = grid_search.best_estimator_

print("Melhores hiperparâmetros encontrados:")
print(grid_search.best_params_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
